#### 一. 同步容器
1. 同步容器是JDK1.2中的代码, 如Vector,HashTable, 这些类本身的方法并没有进行同步, 而是需要通过`Collections.synchronizedXxx`方法来创建出封装后的同步类
2. 由于Vector,HashTable在设计之处并没有对其内部方法或组合方法进行同步, 其某些组合操作可能会发生异常, 因此, 引入`ConcurrentModifictionExeption`来表示操作中出现了异常:  
  例如: 一个线程正在迭代Vector, 另一个线程却删除了Vector的最后一个元素, 此时迭代的线程如果继续则最后会报"ArrayOutofBoundsExeption", 因此, Vector内部的迭代器的next方法, 会先判断Vector的长度是否发生了变化: 
    ```java
    final void checkForComodification() {
        if (modCount != expectedModCount)
            throw new ConcurrentModificationException();
    }
    ```
3. 由于foreach迭代在javac中仍然会被翻译成迭代器迭代, 因此Vector的foreach仍然会报"并发修改异常"
4. "并发修改异常"的出现是因为有些时候, 开发人员并不希望在迭代期间对容器加锁.  
   比如: Vector很大, 如果所有操作都要等待迭代后进行, 则会降低吞吐率. 一种解决办法是对容器先克隆再迭代( 并发的CopyOnWriteArrayList没有在迭代时复制数组,解决办法在下)
   
   
#### 二. 并发容器
1. CopyOnWriteArrayList  
    * (1) 首先, CopyOnWriteArrayList的add方法, 是在对数组进行 Arrays.copyOf形成副本后(副本的内存地址和原数组不通), 在副本上进行添加的, 添加完毕后, 将这个副本赋值给内部的数组  
    * (2) 其次, CopyOnWriteArrayList的迭代器, 先将内部的数组传递给自身的参数, 即使此时有添加操作完毕, 但迭代器仍然指向的是旧的数组的内存地址, 并不影响迭代; 因此, CopyOnWriteArrayList在迭代时并不需要加锁或是检查并发修改异常  
2. ConcurrentHashMap(1.7)  
    * (3) ConcurrentHashMap支持多达16个并发写入  
     ConcurrentHashMap使用了所分段技术-Lock Striping; 其内部实现了一个长达16个锁的数组, 每个锁保护所有散列桶的1/16, 其中第n个散列桶由第(N mod 16)个锁进行保护  
    * (4) ConcurrentHashMap在执行rehash时的难度更大    
    由于ConcurrentHashMap由16个锁进行保护, 因此在执行对整个map进行的操作类型上, 只有当16个锁全部占到, 才可能完成操作  
    * (5) 一些在整个map上进行计算的方法, 如size/isEmpty. 这些方法的语义被削弱, 返回的是一个近似值, 而不是精确值; 因为这些操作在并发环境下的用处很小, 因为它的返回值总在不断变化 
2. ConcurrentHashMap(1.8)
    * (1) 初始化ConcurrentHashMap时, 如果设置哈希桶个数为n, 则实际取值为大于n的最小2的m次幂  
    * (2) 1.8里的ConcurrentHashMap的put操作, 由分段锁变为synchronized + cas操作. 当哈希表要插入1个节点时, 由于 ConcurrentHashMap采用"独立"链法进行冲突解决, 在形同桶上冲突的节点形成单链表, 因此插入时尝试在该链表的结尾插入, 插入时synchronized的锁为该链表的头结点, 即实现了更细粒度的锁分离. 插入成功后, 通过cas操作更新整个哈希表的节点个数.  
        * "独立链"的散列冲突在1.8的一个优化是: 如果冲突链表的长度>8. 会转变成红黑树的结构进行插入, 从O(N)降低为O(logN)